<a href="https://colab.research.google.com/github/CurtCalledBurt/DS-Unit-2-Kaggle-Challenge/blob/master/module3/assignment_kaggle_challenge_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 3


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [0]:
should_not_have_zeroes_or_close_to_zeroes = ['amount_tsh', 'longitude', 
                                             'latitude', 'num_private',
                                             'construction_year']

should_nots = should_not_have_zeroes_or_close_to_zeroes

def wrangle(data):
  import numpy as np
  import pandas as pd
  
  # Prevents 'SettingWithCopyWarning' whatever that is
  data = data.copy()
  
  # input NaN's instead of zeroes and close to zero values where there should be NaN's
  for col in should_not_have_zeroes_or_close_to_zeroes:
    data[col] = np.where(data[col].between(-0.0001, 0.0001), np.nan, data[col])
  
  # convert days of year to datetime
  data['date_recorded'] = pd.to_datetime(data['date_recorded'])
  #OrdinalEncoder seems to not like datetime, so we drop all information except the year of date_recorded
  data['date_recorded'] = data['date_recorded'].dt.year
  
  # creat a feature of time between last inspection and construction
  data['time_between_inspections'] = data['date_recorded'] - data['construction_year']
  
  return(data)

In [0]:
train = wrangle(train)
test = wrangle(test)

In [0]:
target = 'status_group'
features = train.columns.drop([target])

X_train = train[features]
y_train = train[target]

X_test = test[features]
# no y_test dataframe b/c we don't have those labels
# y_test = test[target]

In [0]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import category_encoders as ce


pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_estimators=100, random_state=2019, n_jobs=-1)
)

In [11]:
pipeline.fit(X_train, y_train)

print('Train Accuracy: ', pipeline.score(X_train, y_train))

Train Accuracy:  0.99996632996633


In [0]:
from sklearn.model_selection import cross_val_score

k=3
scores = cross_val_score(pipeline, X_train, y_train, 
                        cv = k, 
                        scoring=None)
# scoring=None isn't necessary, you could leave it out. It just tells cross_val_score
# to score using the model's default scoring method

In [14]:
scores

array([0.80681818, 0.80757576, 0.8059596 ])

In [25]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform


param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'],
    'randomforestclassifier__min_samples_leaf': randint(1, 50),
    'randomforestclassifier__n_estimators': randint(50, 200),
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None],
    'randomforestclassifier__max_features': uniform(0, 1)  
}

search_param = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=10,
    cv=5,
    return_train_score=True,
    verbose=10,
    n_jobs=-1
)

search_param.fit(X_train, y_train);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   20.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 13.5min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 16.9min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 18.2min finished


In [26]:
print('Best hyperparameters', search_param.best_params_)
print('Cross-validation Score', search_param.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': 15, 'randomforestclassifier__max_features': 0.6090297513827546, 'randomforestclassifier__min_samples_leaf': 20, 'randomforestclassifier__n_estimators': 136, 'simpleimputer__strategy': 'median'}
Cross-validation Score 0.7855218855218855


In [27]:
pd.DataFrame(search_param.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestclassifier__max_depth,param_randomforestclassifier__max_features,param_randomforestclassifier__min_samples_leaf,param_randomforestclassifier__n_estimators,param_simpleimputer__strategy,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
7,68.491120,0.327388,0.864774,0.072911,15,0.60903,20,136,median,"{'randomforestclassifier__max_depth': 15, 'ran...",0.786297,0.782089,0.785774,0.783670,0.789779,0.785522,0.002608,1,0.809866,0.812012,0.811574,0.812395,0.813034,0.811776,0.001069
1,93.658983,1.130380,0.985174,0.043152,15,0.684606,24,164,mean,"{'randomforestclassifier__max_depth': 15, 'ran...",0.784193,0.781247,0.785017,0.780724,0.788096,0.783855,0.002685,2,0.807277,0.807719,0.807976,0.808123,0.807710,0.807761,0.000288
3,15.906108,0.245973,0.632160,0.104564,None,0.219155,30,80,mean,"{'randomforestclassifier__max_depth': None, 'r...",0.780490,0.776534,0.781481,0.778283,0.784896,0.780337,0.002857,3,0.799112,0.800480,0.798106,0.801242,0.801734,0.800135,0.001346
4,55.843328,0.590807,0.686899,0.101813,15,0.781165,31,87,median,"{'randomforestclassifier__max_depth': 15, 'ran...",0.777460,0.776366,0.779714,0.777441,0.786243,0.779444,0.003570,4,0.797008,0.798460,0.799621,0.799621,0.800051,0.798952,0.001107
9,42.263103,9.700817,0.536434,0.135938,10,0.419022,1,154,mean,"{'randomforestclassifier__max_depth': 10, 'ran...",0.761047,0.758017,0.761700,0.758249,0.765365,0.760875,0.002681,5,0.779478,0.781056,0.783186,0.782870,0.781322,0.781582,0.001342
8,24.945922,0.193296,0.347344,0.043361,10,0.55634,30,64,median,"{'randomforestclassifier__max_depth': 10, 'ran...",0.753219,0.748422,0.755135,0.752189,0.758461,0.753485,0.003313,6,0.760917,0.761611,0.764078,0.763321,0.760932,0.762172,0.001295
2,42.010632,1.081902,0.526389,0.100789,10,0.937761,46,68,median,"{'randomforestclassifier__max_depth': 10, 'ran...",0.750694,0.746318,0.751515,0.747306,0.758629,0.750892,0.004337,7,0.756876,0.758412,0.759512,0.758396,0.757249,0.758089,0.000938
5,9.116763,0.273380,0.629734,0.138324,10,0.0577847,40,154,mean,"{'randomforestclassifier__max_depth': 10, 'ran...",0.739163,0.732514,0.740657,0.735859,0.743475,0.738333,0.003808,8,0.741135,0.744650,0.743855,0.744318,0.742477,0.743287,0.001306
6,40.459571,0.190575,0.466617,0.064182,5,0.600666,29,176,median,"{'randomforestclassifier__max_depth': 5, 'rand...",0.724097,0.716186,0.715067,0.718434,0.727648,0.720286,0.004821,9,0.722553,0.725857,0.719550,0.720391,0.719730,0.721616,0.002375
0,7.255048,0.079424,0.372066,0.080023,5,0.17022,15,88,mean,"{'randomforestclassifier__max_depth': 5, 'rand...",0.718037,0.713324,0.722475,0.720960,0.725543,0.720067,0.004151,10,0.719460,0.722616,0.723274,0.720918,0.719330,0.721120,0.001605


In [0]:
pipeline_best = search_param.best_estimator_

y_pred = pipeline_best.predict(X_test)

submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('Curtis-McKendrick-Pipes-RandomForestCV.csv', index=False)